### Steps to deploy a minimal ML model with prefect and coiled.  
#### Pramble 
In this example we will use anaconda for our python enviroment manager.  Other python enviroments can also be used, but there is an extra step in the python enviroment installation process: `dask` and `dask-distributed` need to be installed seperately when using pip.

Furthermore you will need to sign up for a coiled account and prefect account.  As of this post, (2021-05-04), `coiled` is free. And for these purposes so is `prefect`.


 
#### 1. In the shell create a new enviroment with `prefect`, `dask`, `coiled`. 

In [ ]:
`conda create --name py38Pcoiled python=3.8 dask coiled prefect dask-ml -c conda-forge -c conda`

- If you are wondering why the python version is being typed out in specifity.  It is because the local and cloud enviroment need to match exactly.  Otherwise all sort of cloud pickle errors occur.  In a later step we will add versions for a couple of other crucial packeges .e.g. `coiled`

#### 2. Add your coiled token

a. In the`coiled` UI navigate to the **Dashboard** -> **Quick Sart**

![Coiled Token](assets/coiled-login.png)

b. Copy `coiled login --token ·········································` to the shell and run.



#### 3. Create an enviroment requirements file.  

There are four options to specify the python enviroment to run the code. Conda, Pip, container and post_build.   It's possible to use all four or any combination:  Yes you are reading this correctly.  There is no need for a dockerfile.
You can name it whatever you want. 

In this case we will use a conda enviroment

In [ ]:
conda env export > <py38mlflow.yml>







It's mot that coiled doesnt use one to execute the flow, but rather it merges your specifications with theirs.  The 






#### 4. Create a software enviroment to run in the cloud

- 


In [7]:
import coiled

In [7]:

import coiled
coiled.create_software_environment(
    # name is the software enviroment name in coiled.  i.e : "<account>/<name>"
    name="gryBox/py38Pcoiled", 
    conda="/home/ilivni/demos/coiled-prefect-demo/py38mlflow.yml",
    pip=["git+https://github.com/grybox/demos.git#subdirectory=coiled-prefect-demo"]
    
)


Updating software environment...
Creating new software environment
Creating new ecr build
STEP 1: FROM coiled/default:sha-050a418
STEP 2: COPY environment.yml environment.yml
--> 65f8ccc690e
STEP 3: RUN conda env update -n coiled -f environment.yml     && rm environment.yml     && conda clean --all -y     && echo "conda activate coiled" >> ~/.bashrc
Solving environment: ...working... done

yaml-0.2.5           | 82 KB     | ########## | 100%
numpy-1.20.2         | 5.8 MB    | ########## | 100%
distributed-2021.4.1 | 1.1 MB    | ########## | 100%
zict-2.0.0           | 10 KB     | ########## | 100%
ncurses-6.2          | 985 KB    | ########## | 100%
urllib3-1.26.4       | 99 KB     | ########## | 100%
traitlets-5.0.5      | 81 KB     | ########## | 100%
pillow-8.1.2         | 678 KB    | ########## | 100%
pendulum-2.1.2       | 72 KB     | ########## | 100%
bokeh-2.3.1          | 8.2 MB    | ########## | 100%
pandas-1.2.4         | 12.0 MB   | ########## | 100%
dask-core-2021.4.1   | 7

In [6]:

import coiled
coiled.create_software_environment(
    # name is the software enviroment name in coiled.  i.e : "<account>/<name>"
    name="gryBox/py38Pcoiled-tst-git", 
    # conda="/home/ilivni/demos/coiled-prefect-demo/py38mlflow.yml",
    pip=["git+https://github.com/grybox/demos.git#subdirectory=coiled-prefect-demo"]
    #https://github.com/gryBox/demos.git
)


Updating software environment...
Creating new software environment
Creating new ecr build
STEP 1: FROM coiled/default:sha-050a418
STEP 2: COPY environment.yml environment.yml
--> 319a53505fa
STEP 3: RUN conda env update -n coiled -f environment.yml     && rm environment.yml     && conda clean --all -y     && echo "conda activate coiled" >> ~/.bashrc
Solving environment: ...working... done

ncurses-6.2          | 817 KB    | ########## | 100%
pip-21.0.1           | 1.8 MB    | ########## | 100%
ld_impl_linux-64-2.3 | 568 KB    | ########## | 100%
readline-8.1         | 362 KB    | ########## | 100%
libgcc-ng-9.1.0      | 5.1 MB    | ########## | 100%
zlib-1.2.11          | 103 KB    | ########## | 100%
tk-8.6.10            | 3.0 MB    | ########## | 100%
setuptools-52.0.0    | 714 KB    | ########## | 100%
sqlite-3.35.4        | 981 KB    | ########## | 100%
python-3.8.8         | 50.0 MB   | ########## | 100%
_libgcc_mutex-0.1    | 3 KB      | ########## | 100%
libstdcxx-ng-9.1.0   | 3

In [ ]:
from ml_flow import flow

In [10]:
from prefect.executors import DaskExecutor
from src.ml_flow import flow
executor = DaskExecutor(
    cluster_class=coiled.Cluster,
    cluster_kwargs={
        "software": "grybox/py38pcoiled",
        "shutdown_on_close": False,
        "name": "prefect-play",
    },
)


state = flow.run(
    executor=executor
)

[2021-05-10 14:45:18-0500] INFO - prefect.FlowRunner | Beginning Flow run for 'ml-flow'
[2021-05-10 14:45:18-0500] INFO - prefect.DaskExecutor | Creating a new Dask cluster with `coiled.cluster.Cluster`...


Output()

[2021-05-10 14:46:19-0500] INFO - prefect.DaskExecutor | The Dask dashboard is available at http://ec2-3-142-53-176.us-east-2.compute.amazonaws.com:8787
[2021-05-10 14:46:50-0500] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


In [18]:
train_mode_tsk = flow.get_tasks(name="Train Model")
train_mode_tsk

[<Task: Train Model>]

In [19]:
taxi_est = state.result[train_mode_tsk[0]]._result.value
taxi_est

LinearRegression()